<a href="https://colab.research.google.com/github/manujsinghwal/chama-data-transformation/blob/main/chama_data_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Chama** is a relatively new, modern company, with an IOS App and Android App. To grow our company we must make a special effort in collecting all information available around the App and make it available for everyone in the company. Some information about App usage is generated by our App and our backend API. Other sources of information, like the Google Play Store, can provide very useful insights on App performance and user rating. Chama is very data-oriented and, most decisions are made based on appropriate metrics, therefore, data quality is a must and a concern of everyone involved.

# Data Transformation
Some event data come as json files and need some transformation to be structured as tables. Convert the [case.json](https://drive.google.com/file/d/1Rrse9d1600E3wM8goEpAQDA-b64D9Eml/view?usp=sharing) file to 3 tables, using Python, with the following rules:

```
Table1:
        CuratedOfferOptions
Columns:
        CurationProvider: in quotes
        OfferId: in quotes
        DealerId: in quotes
        UniqueOptionId: in quotes
        OptionId: in quotes
        IsMobileDealer: without quotes
        IsOpen: without quotes
        Eta: in quotes
        ChamaScore: without quotes
        ProductBrand: in quotes
        IsWinner: without quotes
        MinimumPrice: without quotes
        MaximumPrice: without quotes
        DynamicPrice: without quotes
        FinalPrice: without quotes
        DefeatPrimaryReason: in quotes
        DefeatReasons: in quotes
        EnqueuedTimeSP: DD/MM/YYYY (converted to Brasilian timezone - UTC-3)
```

```
Table2:
        DynamicPriceOption
Columns:
        Provider: in quotes
        OfferId: in quotes
        UniqueOptionId: in quotes
        BestPrice: without quotes
        EnqueuedTimeSP: DD/MM/YYYY (converted to Brasilian timezone - UTC-3)
```

```
Table3:
        DynamicPriceRange
Columns:
        Provider: in quotes
        OfferId: in quotes
        MinGlobal: without quotes
        MinRecommended: without quotes
        MaxRecommended: without quotes
        DifferenceMinRecommendMinTheory: without quotes
        EnqueuedTimeSP: DD/MM/YYYY (converted to Brasilian timezone - UTC-3)
```

# Introduction and Data Exploration
Before going through the actual data transformation, in this section, we will spend some time getting familiar with the provided data. This will enable us to better understand the problem that the company is facing, and we could come up with a satisfactory solution.

In [ ]:
# Mounting Google drive as case.json file stored in a drive directory
from google.colab import drive
drive.mount('/content/gdrive')

In [33]:
# Importing pandas for data exploration and transformation
import pandas as pd

In [34]:
# Reading json file from Google drive directory
df = pd.read_json("gdrive/My Drive/Colab Notebooks/Dataset/case.json")

# Showing default first 5 records
df.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 08:04:08 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 11:43:23 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 09:04:04 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
3,2021-08-25 05:02:55 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
4,2021-09-05 08:03:28 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."


In [36]:
# Checking the total rows and columns available in .json file.
df.shape

(37, 3)

We have 37 rows in the DataFrame, and 3 columns:

*   `EnqueuedTimeUTC` - Timestamp of when the event occurred, in  UTC timezone.
*   `EventName` - The name of the event that occurred.
*   `Payload` - Extra info that is event-specific.

The first thing that we observe in the sample above is that different `EventName` values correspond to the tables that are requested. We will probably need to partition the DataFrame by that column.

Before partitioning the event name and going into creating the specific tables, it may be a good decision to transform the timezone now.

In [40]:
# Checking the distinct EventName
df['EventName'].value_counts()

EventName
DynamicPrice_Result    32
CurateOffer_Result      5
Name: count, dtype: int64

We have two events in our data: `DynamicPrice_Result` and `CurrateOffer_Result`.

We will use the rows from the former one to create the second and the third table, and the rows from the latter to create the first table.